In [1]:
import pandas as pd

# Assuming your dataframes are named firstname_sex and df

# Sample data
data_firstname_sex = {
    'firstname': ['marie', 'jean', 'pierre'],
    'male': [10145, 1869615, 1475841],
    'female': [2390322, 6476, 5047]
}

data_df = {
    'firstname': ['Louis-Jean', 'Marie', 'Marie', 'Marguerite', 'Antoine', 'Louis'],
    'sex': ['homme', 'femme', 'femme', 'femme', 'homme', 'homme']
}

# Create dataframes
firstname_sex = pd.DataFrame(data_firstname_sex)
df = pd.DataFrame(data_df)

# Function to determine predicted sex
def predict_sex(row):
    firstname = row['firstname'].lower()
    match = firstname_sex[firstname_sex['firstname'] == firstname]
    if len(match) > 0:
        if match['male'].iloc[0] > match['female'].iloc[0]:
            return 'homme'
        else:
            return 'femme'
    else:
        return None

# Apply function to create 'pred_sex' column
df['pred_sex'] = df.apply(predict_sex, axis=1)

df

,firstname,sex,pred_sex
0,Louis-Jean,homme,None
1,Marie,femme,femme
2,Marie,femme,femme
3,Marguerite,femme,None
4,Antoine,homme,None
5,Louis,homme,None
